In [13]:
from webpage_parser import *
import pandas as pd
import pickle
import json
import requests
import matplotlib.pyplot as plt
import re

In [16]:
df = pd.read_csv('fake_or_real_news.csv')
df = df.drop('Unnamed: 0', 1)
print(df.shape)
df.head()

(6335, 3)


,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [17]:
r = requests.get('https://raw.githubusercontent.com/stanfordnlp/CoreNLP/master/data/edu/stanford/nlp/patterns/surface/stopwords.txt')
stopwords = set(r.text.split())

In [28]:
def clean_string(string):
    string.strip(',')
    string.strip('.')
    string.strip('?')
    string.strip('/')
    string.strip('!')
    string.strip('$')
    string.strip('#')
    string.strip('^')
    string.strip('&')
    string.strip(':')
    string.strip(';')
    string.strip('-')
    string.strip('=')
    string.strip('"')
    lst = string.split()
    return " ".join(list(filter(lambda x: filter_word(x), lst)))
    

def filter_word(word):
    if 'http' in word or '@' in word:
        return False
    elif '(' in word or ')' in word:
        return False
    elif word in stopwords:
        return False 
    elif len(word) <= 2:
        return False
    return True  

In [30]:
for i in range(len(df['title'])):
    df['title'][i] = clean_string(df['title'][i])
    df['text'][i] = clean_string(df['text'][i])

In [31]:
df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, Shillman Journalism Fellow ...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry Paris gesture sympathy,U.S. Secretary State John Kerry said Monday wi...,REAL
3,Bernie supporters Twitter erupt anger DNC: 'We...,Kaydee King November 2016 The lesson tonight's...,FAKE
4,The Battle New York: Why This Primary Matters,It's primary day New York front-runners Hillar...,REAL


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix

In [34]:
count_vect = CountVectorizer()

text_counts = count_vect.fit_transform(df['text'])
title_counts = count_vect.fit_transform(df['title'])

In [38]:
text_counts.shape

(6335, 65653)

In [36]:
title_counts.shape

(6335, 9996)